# Part 3 of a 3-part assignment for Week 3 on Clustering of Toronto neighborhoods 
## Steps
 - ### 1. Dataframe with latitude and longitude coordinates for the neighorhood from selected postal codes taken from Part 2
 - ### 2. folium used for world map with Toronto as center
 - ### 3. imposed Toronto neighborhoods on the map 

#### Import libraries and install Beautifulsoup4

In [2]:
import pandas as pd
import sys
sys.path.append('C:\\Python27\\Scripts\\pip.exe')
!pip install beautifulsoup4

!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup
import requests
import urllib.request
from lxml import html

#### Scrape the web page containing Canadian postal codes, boroughs and neighborhoods

In [3]:
# Scrape the page with BeautifulSoup

quote_page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(quote_page)
#print(page.status_code)

soup= BeautifulSoup(page.text, "html.parser")

right_table=soup.find('table', {"class" : 'wikitable sortable'})
#print(right_table)

# Transfer to a DataFrame 
df = pd.read_html(str(right_table))[0]
df_all = pd.DataFrame(df)
#print(df_all.shape)
#df_all.head(5)

#### Get the dataframe after filtering out "Not assigned" boroughs and neighborhoods etc., restricting in the postal codes selected  

In [4]:
# Data shaping: filtered out "Not assigned" etc.

df.drop(  df[ df['Borough']=='Not assigned'].index, inplace=True)
restr = df['Neighborhood']=='Not assigned'
post_lst=['M5G','M2H', 'M4B','M1J','M4G','M4M', 'M1R','M9V','M9L','M5V','M1B','M5A']  # Custom list of postal codes for Toronto

df.replace(to_replace=list(df[restr]['Neighborhood']), value= list(df[restr]['Borough']), inplace=True)

df_filtered = df[df['Postcode'].isin(post_lst)].reset_index(drop=True)

df_filtered.head()

,Postcode,Borough,Neighborhood
0,M5A,Downtown Toronto,Harbourfront
1,M1B,Scarborough,Rouge
2,M1B,Scarborough,Malvern
3,M4B,East York,Woodbine Gardens
4,M4B,East York,Parkview Hill


#### Retrival of Longitude and Latitude for the postal code areas


In [8]:
# Part 2 
path='http://cocl.us/Geospatial_data'
df_geosp = pd.read_csv(path)
df_geosp.rename(columns={'Postal Code':'Postcode'}, inplace=True)

neighborhoods=pd.merge(df_filtered, df_geosp, on='Postcode')
#print(neighborhoods.head())

pd.DataFrame(neighborhoods.reset_index(drop=True)).head()
#neighborhoods

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M1B,Scarborough,Rouge,43.806686,-79.194353
2,M1B,Scarborough,Malvern,43.806686,-79.194353
3,M4B,East York,Woodbine Gardens,43.706397,-79.309937
4,M4B,East York,Parkview Hill,43.706397,-79.309937


In [9]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(len(neighborhoods['Borough'].unique()),\
                                len(neighborhoods['Neighborhood'].unique()) )  )

The dataframe has 6 boroughs and 28 neighborhoods.


### Map of Toronto neighborhoods

In [10]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
#import json 
import numpy as np
import pandas as pd

<a id='item1'></a>

#### Toronto neighborhoods map with given list of postal codes  

In [11]:
get_ipython().magic(u'matplotlib inline')

# create map of Toronto Neighbourhoods (FSAs) using retrived latitude and longitude values
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=10)

# Toronto using latitude and longitude values for those neighborhoods

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Select those borough with word Toronto in its name  

In [12]:
# Select those borough with word Toronto in its name  
neighborhoods=neighborhoods[neighborhoods['Borough'].str.contains('Toronto')]
neighborhoods.reset_index(drop=True)


,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
1,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
2,M4M,East Toronto,Studio District,43.659526,-79.340923
3,M5V,Downtown Toronto,CN Tower,43.628947,-79.394420
4,M5V,Downtown Toronto,Bathurst Quay,43.628947,-79.394420
5,M5V,Downtown Toronto,Island airport,43.628947,-79.394420
6,M5V,Downtown Toronto,Harbourfront West,43.628947,-79.394420
7,M5V,Downtown Toronto,King and Spadina,43.628947,-79.394420
8,M5V,Downtown Toronto,Railway Lands,43.628947,-79.394420
9,M5V,Downtown Toronto,South Niagara,43.628947,-79.394420


In [13]:
get_ipython().magic(u'matplotlib inline')

# create map of Toronto Neighbourhoods (FSAs) using retrived latitude and longitude values
# Toronto using latitude and longitude values for those neighborhoods

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Thanks
## End of Part 3 of 3

<a id='item4'></a>